In [1]:
import sqlite3
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from utils.dummy import dummy_data
from utils.utils import get_citation_doi
from utils.utils import get_paper_citation_pairs
from acm_api.acm import acm_meta
from evaluation.evaluate import Train_Test_Split_Plain
from cf.cf import CF
from cf.cf_unit import CFU
from cf.cf_rank import CFUR

<h2>Processing the input file with doi of papers</h2>

In [2]:
doi_list = []
data = {}

# reading the input file with doi of papers
with open('doi/doi.txt', 'r') as doi_file:
    lines = doi_file.readlines()
    
for line in lines:
    doi_list.append(line.strip())
    
con = sqlite3.connect('fake.db')
cur = con.cursor()

is_table_exists = cur.execute(
    """
    SELECT name FROM sqlite_master WHERE type='table'
    AND name='PAPERS'; """).fetchall()

if is_table_exists == []:
    cur.execute(""" 
        CREATE TABLE PAPERS (doi TEXT, data TEXT);
    """)
    
print('processing doi....')
for doi in tqdm(doi_list):
    print(doi)
    cursor = con.execute("SELECT * FROM PAPERS WHERE doi=?;", (doi,))
    fetch_data = cursor.fetchone()
    
    if fetch_data is None:
        temp = acm_meta(doi)
        insert_statement = 'INSERT INTO PAPERS(doi, data) VALUES(?, ?)'
        
        con.execute(insert_statement, (doi, str(temp)))
        data[doi] = get_citation_doi(temp)
        
    else:
        data[doi] = get_citation_doi(fetch_data[1])

con.commit()
con.close()

processing doi....


  0%|          | 0/30 [00:00<?, ?it/s]

1
extracting citation's doi....


  0%|          | 0/70 [00:00<?, ?it/s]

2
extracting citation's doi....


...

  0%|          | 0/70 [00:00<?, ?it/s]

29
extracting citation's doi....


  0%|          | 0/70 [00:00<?, ?it/s]

30
extracting citation's doi....


  0%|          | 0/70 [00:00<?, ?it/s]

In [3]:
# Paper's doi and it's citations
# data

<h2>Code for creating the rating matrix</h2>

In [4]:
citation_set = set()
for key in data.keys():
    citation_set.update(data[key])

index_ = data.keys()
df = pd.DataFrame(np.zeros((len(index_), len(citation_set))), index=index_, columns=citation_set)

for i in index_:
    for j in citation_set:
        if j in data[i]:
            df.loc[i][j] = 1

df.to_csv('citation-web/matrix-way/citation-matrix.csv')

<h2>Creating the citation pair</h2>

In [5]:
citation_pair_path = get_paper_citation_pairs(df)

writing citation pairs....


  0%|          | 0/30 [00:00<?, ?it/s]

In [6]:
########################################
	# Experiment 1
	# Plain CF
	
	
	########################################


	########################################
	# Experiment 1
	# Plain CF
	
	# TO DO
	########################################

In [7]:
evaluate = Train_Test_Split_Plain(CFU, min_ref_limit=5, train_size=0.5)
evaluate.fit(data)

The following doi's are ignored because of low reference :
{}
writing citation pairs....


  0%|          | 0/30 [00:00<?, ?it/s]

removing the indexes with no citations....
normalizing dataframe....


100%|█████████████████████████████████████████| 30/30 [00:00<00:00, 3163.28it/s]


creating similarity matrix....


...

computing recommendations for : 1


100%|██████████████████████████████████████████| 34/34 [00:00<00:00, 300.97it/s]

finishing evaluation....


  0%|          | 0/15 [00:00<?, ?it/s]

0.2510189437631056

In [8]:
evaluate = Train_Test_Split_Plain(CF, min_ref_limit=5, train_size=0.5)
evaluate.fit(data)

The following doi's are ignored because of low reference :
{}
writing citation pairs....


  0%|          | 0/30 [00:00<?, ?it/s]

Creating similarity matrix....


100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 81.53it/s]


Computing recommendations for : 23


...

Computing recommendations for : 1


100%|██████████████████████████████████████████| 34/34 [00:00<00:00, 301.39it/s]

finishing evaluation....


  0%|          | 0/15 [00:00<?, ?it/s]

0.25423451728808616

In [9]:
evaluate = Train_Test_Split_Plain(CFUR, min_ref_limit=5, train_size=0.5)
evaluate.fit(data)

The following doi's are ignored because of low reference :
{}
writing citation pairs....


  0%|          | 0/30 [00:00<?, ?it/s]

removing the indexes with no citations....
page rank normalization on dataframe....


100%|█████████████████████████████████████████| 30/30 [00:00<00:00, 5659.31it/s]


creating similarity matrix....


...

Computing recommendations for : 1


0it [00:00, ?it/s]

finishing evaluation....


  0%|          | 0/15 [00:00<?, ?it/s]

0.0

In [10]:
x = evaluate.recommender(default_path='temp/', file_='matrix-way/temp-citation-matrix.csv',file_pair='pair-way/temp-citation-pairs.txt')

removing the indexes with no citations....
page rank normalization on dataframe....


100%|█████████████████████████████████████████| 30/30 [00:00<00:00, 5297.40it/s]


creating similarity matrix....


100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 80.55it/s]


normalizing the similarity matrix....


100%|███████████████████████████████████████| 100/100 [00:00<00:00, 3873.72it/s]


In [11]:
x.similarity_mat

,287,265,229,296,235,222,227,261,255,256,...,241,248,273,200,215,220,237,263,286,247
287,0.000000,0.011656,0.011634,0.012162,0.011132,0.010808,0.012164,0.011219,0.010119,0.010345,...,0.011673,0.010915,0.011792,0.011399,0.011944,0.010813,0.011342,0.011365,0.010296,0.011095
265,0.010625,0.000000,0.009521,0.010148,0.009447,0.009434,0.009954,0.011017,0.010121,0.011287,...,0.010055,0.011208,0.010936,0.009328,0.007819,0.010323,0.010055,0.010628,0.011234,0.010761
229,0.011281,0.010127,0.000000,0.009697,0.010961,0.011831,0.010304,0.009942,0.009671,0.009126,...,0.010889,0.010040,0.011679,0.011142,0.010792,0.011273,0.010347,0.010720,0.012881,0.010282
296,0.010550,0.009656,0.008675,0.000000,0.009931,0.009298,0.009593,0.009652,0.011607,0.011867,...,0.009866,0.010310,0.009468,0.009807,0.010961,0.008527,0.008945,0.009311,0.005905,0.011313
235,0.010381,0.009664,0.010542,0.010677,0.000000,0.010369,0.010536,0.010764,0.008989,0.008483,...,0.011134,0.009582,0.011313,0.010632,0.011035,0.011527,0.009824,0.011538,0.008780,0.010513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,0.009678,0.010136,0.010406,0.008799,0.011063,0.010271,0.008500,0.011289,0.008485,0.009787,...,0.010303,0.009332,0.010547,0.011151,0.009793,0.000000,0.008718,0.012101,0.012662,0.009648
237,0.009929,0.009656,0.009342,0.009027,0.009222,0.009298,0.010465,0.009652,0.009673,0.009128,...,0.009866,0.010310,0.010145,0.008172,0.010047,0.008527,0.000000,0.009311,0.010629,0.009899
263,0.011361,0.011656,0.011053,0.010731,0.012368,0.010808,0.011404,0.012341,0.010119,0.011141,...,0.012287,0.010272,0.011202,0.010687,0.011148,0.013516,0.010633,0.000000,0.011325,0.011095
286,0.007503,0.008980,0.009680,0.004961,0.006861,0.008243,0.006326,0.007779,0.009355,0.009932,...,0.007668,0.008013,0.008994,0.005928,0.005521,0.010309,0.008847,0.008255,0.000000,0.005984


In [12]:
dummy_data()